In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
twitter = pd.read_csv("final_dataset.csv")    

Wall time: 5.4 s


In [3]:
pd.set_option('display.max_rows', 1000)
twitter

_score  tweet_id                                               text  \
0           391  0x376b20  people who post add me on snapchat must be deh...   
1           433  0x2d5350  brianklaas as we see trump is dangerous to fre...   
2           376  0x1cd5b0                    now issa is stalking tasha 😂😂😂    
3           120  0x1d755c  riskshow thekevinallison thx for the best time...   
4          1021  0x2c91a8            still waiting on those supplies liscus    
...         ...       ...                                                ...   
1451386      94  0x321566  im so happy nowonder the name of this show hap...   
1451387     627  0x38959e  in every circumtance id like to be thankful to...   
1451388     274  0x2cbca6  theres currently two girls walking around the ...   
1451389     840  0x24faed  ah corporate life where you can date  using ju...   
1451390     360  0x34be8c                  blessed to be living sundayvibes    

        identification       emotion  \
0                train  anticipation   
1                train       sadness   
2                train          fear   
3                train           joy   
4                train  anticipation   
...                ...           ...   
1451386          train           joy   
1451387          train           joy   
1451388          train           joy   
1451389          train           joy   
1451390          train           joy   

                                              text_stemmed  Category  
0        peopl who post add me on snapchat must be dehy...         4  
1        brianklaa as we see trump is danger to freepre...         8  
2                              now issa is stalk tasha 😂😂😂         1  
3        riskshow thekevinallison thx for the best time...         7  
4                         still wait on those suppli liscu         4  
...                                                    ...       ...  
1451386  im so happi nowond the name of thi show happi ...         7  
1451387  in everi circumt id like to be thank to the al...         7  
1451388  there current two girl walk around the librari...         7  
1451389  ah corpor life where you can date use just the...         7  
1451390                         bless to be live sundayvib         7  

[1451391 rows x 7 columns]

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(twitter.text_stemmed,twitter.emotion,
                                                test_size=0.2, random_state = 42)

In [5]:
twitter[:10]

_score  tweet_id                                               text  \
0     391  0x376b20  people who post add me on snapchat must be deh...   
1     433  0x2d5350  brianklaas as we see trump is dangerous to fre...   
2     376  0x1cd5b0                    now issa is stalking tasha 😂😂😂    
3     120  0x1d755c  riskshow thekevinallison thx for the best time...   
4    1021  0x2c91a8            still waiting on those supplies liscus    
5     481  0x368e95                           love knows no gender 😢😭    
6     827  0x249c0c  dstvngcare dstvng more highlights are being sh...   
7     631  0x359db9  the ssm debate  a manufactured fantasy used to...   
8     839  0x23b037  i love suffering 🙃🙃 i love when valium does no...   
9     560  0x1fde89  can someone tell my why my feeds scroll back t...   

  identification       emotion  \
0          train  anticipation   
1          train       sadness   
2          train          fear   
3          train           joy   
4          train  anticipation   
5          train           joy   
6          train       sadness   
7          train  anticipation   
8          train           joy   
9          train         anger   

                                        text_stemmed  Category  
0  peopl who post add me on snapchat must be dehy...         4  
1  brianklaa as we see trump is danger to freepre...         8  
2                        now issa is stalk tasha 😂😂😂         1  
3  riskshow thekevinallison thx for the best time...         7  
4                   still wait on those suppli liscu         4  
5                             love know no gender 😢😭         7  
6  dstvngcare dstvng more highlight are be shown ...         8  
7  the ssm debat a manufactur fantasi use to dist...         4  
8  i love suffer 🙃🙃 i love when valium doe noth t...         7  
9  can someon tell my whi my feed scroll back to ...         3

In [6]:
%%time
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

BOW = TfidfVectorizer(tokenizer=nltk.word_tokenize, 
                      stop_words='english', 
                      #min_df=2,
                      max_df = 0.6, 
                      max_features = 20000)
                      
x_train = BOW.fit_transform(x_train)
x_test = BOW.transform(x_test)

term_weight = np.asarray(x_train.mean(axis=0)).ravel().tolist()
term_array = pd.DataFrame({'term': BOW.get_feature_names(), 'weight': term_weight})
term_array.sort_values(by='weight', ascending=False, inplace=True)
term_array

Wall time: 2min 40s


term    weight
16717         thi  0.015838
9751         love  0.015443
9445         life  0.014071
16520       thank  0.012985
3938          day  0.012614
...           ...       ...
19374       👋🏾•••  0.000005
4310   diltotgaya  0.000005
19317           🎲  0.000004
19182           ⠀  0.000004
19093           ♣  0.000003

[20000 rows x 2 columns]

In [8]:
"😕" in BOW.get_feature_names()

True

In [9]:
type(x_train)

scipy.sparse.csr.csr_matrix

In [10]:
y_train

680793            trust
815757          sadness
1124461        surprise
352702          sadness
352523            trust
               ...     
259178              joy
1414414             joy
131932              joy
671155     anticipation
121958     anticipation
Name: emotion, Length: 1161112, dtype: object

In [11]:
## deal with label (string -> one-hot)
import keras
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    #return keras.utils.to_categorical(enc)
    return keras.utils.np_utils.to_categorical(enc)   #Allison/Moo said so, because of version

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 680793        trust
815757      sadness
1124461    surprise
352702      sadness
Name: emotion, dtype: object

y_train.shape:  (1161112,)
y_test.shape:  (290279,)


## After convert
y_train[0:4]:
 [[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (1161112, 8)
y_test.shape:  (290279, 8)


In [12]:
# I/O check
input_shape = x_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  20000
output_shape:  8


In [13]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 10000
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64,32
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64,32
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 64)                1280064   
                                                                 
 re_lu (ReLU)                (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 re_lu_1 (ReLU)              (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 softmax (Softmax)           (None, 8)                 0     

In [14]:
x_train.sort_indices()
x_test.sort_indices()

In [15]:
from keras.callbacks import CSVLogger
import tensorflow as tf

#csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 5 #25
batch_size = 64 #32, 100

es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max')
callbacks = [es] #Early Stopping

# training!
history = model.fit(x_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=callbacks,
                    validation_data = (x_test, y_test))
print('training finish')

Epoch 1/5


C:\Users\alexc\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


18143/18143 [==============================] - 203s 11ms/step - loss: 1.3200 - accuracy: 0.5214 - val_loss: 1.2773 - val_accuracy: 0.5374
Epoch 2/5
18143/18143 [==============================] - 200s 11ms/step - loss: 1.2245 - accuracy: 0.5548 - val_loss: 1.2720 - val_accuracy: 0.5409
training finish


In [16]:
pred_result = model.predict(x_test, batch_size=128) #128
pred_result[:5]

array([[0.14836648, 0.04212337, 0.15480943, 0.03308226, 0.21177396,
        0.26413223, 0.05552765, 0.09018464],
       [0.01731134, 0.30944312, 0.0403496 , 0.03917805, 0.27731386,
        0.12049851, 0.03004997, 0.16585556],
       [0.03144   , 0.03075863, 0.20133059, 0.01250601, 0.27844   ,
        0.34669498, 0.0561654 , 0.04266438],
       [0.04125348, 0.19344606, 0.16691317, 0.01574966, 0.43413737,
        0.08184849, 0.04147959, 0.02517226],
       [0.01214604, 0.05816235, 0.04672039, 0.02222294, 0.25231025,
        0.44147667, 0.07884371, 0.08811763]], dtype=float32)

In [17]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['sadness', 'anticipation', 'sadness', 'joy', 'sadness'],
      dtype=object)

In [18]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), pred_result), 2)))

testing accuracy: 0.54


In [20]:
twitter_test_data = pd.read_csv("twitter_test_data.csv")

In [21]:
%%time
x_test_twitter = BOW.transform(twitter_test_data['text'])
x_test_twitter.sort_indices()

pred_result_test_data = model.predict(x_test_twitter, batch_size=128)

print('x_test.shape: ', x_test_twitter.shape)

x_test.shape:  (411972, 20000)
Wall time: 1min 24s


In [22]:
pred_result_test_data = label_decode(label_encoder, pred_result_test_data)
pred_result_test_data[:5]

array(['joy', 'anticipation', 'joy', 'anticipation', 'trust'],
      dtype=object)

In [23]:
upload_df = pd.DataFrame()
upload_df["id"] = twitter_test_data["tweet_id"]
upload_df["emotion"] = pred_result_test_data
upload_df

id       emotion
0       0x28b412           joy
1       0x2de201  anticipation
2       0x218443           joy
3       0x2939d5  anticipation
4       0x26289a         trust
...          ...           ...
411967  0x2913b4           joy
411968  0x2a980e           joy
411969  0x316b80       sadness
411970  0x29d0cb         anger
411971  0x2a6a4f       sadness

[411972 rows x 2 columns]

In [24]:
upload_df.emotion.nunique()

8

In [26]:
upload_df.to_csv("./uploads/TF-IDF_20k.csv",index=False)